In [44]:
from ortools.graph import pywrapgraph


demand_amounts = [-40500, -22230, -85200, -47500]
dist_capacity = [100000, 100000]

# imbalance and sink node
imbalance = -(sum(demand_amounts)+sum(dist_capacity))
demand_amounts.append(imbalance)

# the total flow from and to nodes
supplies = dist_capacity + demand_amounts
print('supplies', supplies)


n_dist = len(dist_capacity)
n_demand = len(demand_amounts)


start_nodes = [s for s in range(n_dist) for d in range(n_demand)]
end_nodes = [d+len(set(start_nodes)) for s in range(n_dist) for d in range(n_demand)]
n_arcs = len(start_nodes)
print('start nodes', start_nodes)
print('end nodes', end_nodes)

transport_costs = [[52,32,11,69,0], [45,84,76,15,0]]

flat_transport_costs = [transport_costs[s][d] for s in range(n_dist) for d in range(n_demand)]
print('flat_transport_costs', flat_transport_costs)

# Instantiate a SimpleMinCostFlow solver.
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc.
for a in range(n_arcs):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[a], end_nodes[a],
                                                999999999, flat_transport_costs[a])

# Add node supplies.
for s in range(len(supplies)):
    min_cost_flow.SetNodeSupply(s, supplies[s])

    
    
if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
    print('Minimum cost:', min_cost_flow.OptimalCost())
    print('')
    print('  Arc    Flow / Capacity  Cost')
    for i in range(min_cost_flow.NumArcs()):
        cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        print('%1s -> %1s   %3s  / %3s       %3s' % (
            min_cost_flow.Tail(i),
            min_cost_flow.Head(i),
            min_cost_flow.Flow(i),
            min_cost_flow.Capacity(i),
            cost))

supplies [100000, 100000, -40500, -22230, -85200, -47500, -4570]
start nodes [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
end nodes [2, 3, 4, 5, 6, 2, 3, 4, 5, 6]
flat_transport_costs [52, 32, 11, 69, 0, 45, 84, 76, 15, 0]
Minimum cost: 4569920

  Arc    Flow / Capacity  Cost
0 -> 2     0  / 999999999         0
0 -> 3   14800  / 999999999       473600
0 -> 4   85200  / 999999999       937200
0 -> 5     0  / 999999999         0
0 -> 6     0  / 999999999         0
1 -> 2   40500  / 999999999       1822500
1 -> 3   7430  / 999999999       624120
1 -> 4     0  / 999999999         0
1 -> 5   47500  / 999999999       712500
1 -> 6   4570  / 999999999         0


In [58]:
names = ['f1', 'f2', 'f3', 'f4', 'f5',
         'x1', 'x2',
         'dc1','dc2','dc3','dc4','dc5',
        'sink']

factories = range(0, 5)
xds = range(5, 7)
dcs = range(7, 12)
sink = 12

arcs = {} #takes nodes, returns arc_index
a = 0

# factories to xds
for f in factories:
    for xd in xds:
        arcs[(f, xd)] = a
        a+=1
        
#xds to dcs
for f in factories:
    for xd in xds:
        arcs[(f, xd)] = a
        a+=1

# factories to sink
for f in fs:
        arcs[(f, sink)] = a
        a+=1
        
f_to_xd_cost = [[30, 50],
                 [23, 66],
                 [35, 14],
                 [70, 12],
                 [65, 70]]

xd_to_dc_cost = [[12, 25, 22, 40, 41],
                 [65, 22, 23, 12, 15]]

costs={}

# f to xd
for f_ind in range(len(factories)):
    for xd_ind in range(len(xds)):
        costs[(factories[f_ind], xds[xd_ind])]=f_to_xd_cost[f_ind][xd_ind]
        
        
# xd to dc
for xd_ind in range(len(xds)):
    for dc_ind in range(len(dcs)):
        costs[(xds[xd_ind], dcs[dc_ind])]=xd_to_dc_cost[xd_ind][dc_ind]


f_capacity = [200, 300, 100, 150, 220]
xd_flow = [0, 0]
dc_demand =  [-150, -100, -110, -200, -180]
waste = sum(f_capacity) + sum(dc_demand)

supplies = f_capacity + xd_flow + dc_demand + [-waste]
print('supplies', supplies)



# Instantiate a SimpleMinCostFlow solver.
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

## Add each arc.
# fractory to xd
for f in factories:
    for xd in xds:
        min_cost_flow.AddArcWithCapacityAndUnitCost(f,
                                                    xd,
                                                    9999999,
                                                    costs[(f, xd)])
        
# xd to dc
for xd in xds:
    for dc in dcs:
        min_cost_flow.AddArcWithCapacityAndUnitCost(xd,
                                                    dc,
                                                    9999999,
                                                    costs[(xd, dc)])
        
# factories to sink
for f in factories:
    min_cost_flow.AddArcWithCapacityAndUnitCost(f,
                                                sink,
                                                9999999,
                                                0)

for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])


    
if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
    print('Minimum cost:', min_cost_flow.OptimalCost())
    print('')
    print('  Arc    Flow / Capacity  Cost')
    for i in range(min_cost_flow.NumArcs()):
        cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        print('%1s -> %1s   %3s  / %3s       %3s' % (
            names[min_cost_flow.Tail(i)],
            names[min_cost_flow.Head(i)],
            min_cost_flow.Flow(i),
            min_cost_flow.Capacity(i),
            cost))



supplies [200, 300, 100, 150, 220, 0, 0, -150, -100, -110, -200, -180, -230]
Minimum cost: 30220

  Arc    Flow / Capacity  Cost
f1 -> x1    60  / 9999999       1800
f1 -> x2   130  / 9999999       6500
f2 -> x1   300  / 9999999       6900
f2 -> x2     0  / 9999999         0
f3 -> x1     0  / 9999999         0
f3 -> x2   100  / 9999999       1400
f4 -> x1     0  / 9999999         0
f4 -> x2   150  / 9999999       1800
f5 -> x1     0  / 9999999         0
f5 -> x2     0  / 9999999         0
x1 -> dc1   150  / 9999999       1800
x1 -> dc2   100  / 9999999       2500
x1 -> dc3   110  / 9999999       2420
x1 -> dc4     0  / 9999999         0
x1 -> dc5     0  / 9999999         0
x2 -> dc1     0  / 9999999         0
x2 -> dc2     0  / 9999999         0
x2 -> dc3     0  / 9999999         0
x2 -> dc4   200  / 9999999       2400
x2 -> dc5   180  / 9999999       2700
f1 -> sink    10  / 9999999         0
f2 -> sink     0  / 9999999         0
f3 -> sink     0  / 9999999         0
f4 -> sink     0 

In [35]:
distances = [[61, 332, 950, 652, 777,  192, 852, 975],
             [604, 51, 134, 606,  22,  633, 461, 328],
             [499, 241, 233, 395, 884, 794, 787,  94],
             [558, 880, 632, 854, 515, 606, 942, 216],
             [787, 964, 70,  592, 270,  26,  50, 340]]

n_dcs = len(distances)
n_shelters = len(distances[0])

demand = [14000, 8000, 25000, 22000, 20000, 17000, 18500, 23000]

cap=150000
cost_per_dc = 30000000
unit_cost = 2  #dolar cent, per centimile, per kit

from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp

# Instantiate a mixed-integer solver, naming it SolveIntegerProblem.
#solver = pywraplp.Solver('SolveIntegerProblem',
#                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

solver = pywrapcp.Solver('solve')

X = {}
for dc in range(n_dcs):
    for shelter in range(n_shelters):
        X[(dc, shelter)] = solver.IntVar(0, 1, 'x_{dc}_{sh}'.format(dc=dc, sh=shelter))

        
travel_costs_dict = {}
for dc in range(n_dcs):
    for shelter in range(n_shelters):
        travel_costs_dict[(dc, shelter)]= X[(dc, shelter)]*distances[dc][shelter]*unit_cost*demand[shelter]

travel_costs = [travel_costs_dict[(dc, shelter)] for dc in range(n_dcs) for shelter in range(n_shelters)]                   

dc_costs=[]
for dc in range(n_dcs):
    is_open = solver.Max([X[(dc, shelter)] for shelter in range(n_shelters)])
    dc_costs.append(is_open*cost_per_dc)

total_cost = solver.Sum(travel_costs+dc_costs)

## constraints

#one and only one dc per shelter
for shelter in range(n_shelters):
    solver.Add(solver.SumEquality([X[(dc, shelter)] for dc in range(n_dcs)], 1))

#n_dcs == x
x = [solver.Max([X[(dc, shelter)] for shelter in range(n_shelters)]) for  dc in range(n_dcs)] 
#solver.Add(solver.SumEquality(x, 4))
solver.Add(solver.Sum(x) <= 5)
solver.Add(solver.Sum(x) >= 0)


### flatten the X
X_flat = [X[(dc, shelter)] for dc in range(n_dcs) for shelter in range(n_shelters)]


### Ths solution finder
objective = solver.Minimize(total_cost, 1)
decision_builder = solver.Phase(X_flat+x,
                                solver.CHOOSE_FIRST_UNBOUND,
                                solver.ASSIGN_MIN_VALUE)
# Create a solution collector.
collector = solver.LastSolutionCollector()
# Add the decision variables.
collector.Add(X_flat)
#collector.Add(X)
collector.Add(x)
collector.Add(dc_costs)
collector.Add(travel_costs)
  
# Add the objective.
collector.AddObjective(total_cost)
solver.Solve(decision_builder, [objective, collector])

if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    for dc in range(n_dcs):
        for shelter in range(n_shelters):
            print('X_{dc}_{sh} ='.format(dc=dc, sh=shelter),
                  collector.Value(best_solution, X[(dc,shelter)]),
                 ' at a cost of', collector.Value(best_solution, travel_costs_dict[(dc,shelter)]))


Objective value: 116566000
X_0_0 = 0  at a cost of 0
X_0_1 = 0  at a cost of 0
X_0_2 = 0  at a cost of 0
X_0_3 = 0  at a cost of 0
X_0_4 = 0  at a cost of 0
X_0_5 = 0  at a cost of 0
X_0_6 = 0  at a cost of 0
X_0_7 = 0  at a cost of 0
X_1_0 = 0  at a cost of 0
X_1_1 = 0  at a cost of 0
X_1_2 = 0  at a cost of 0
X_1_3 = 0  at a cost of 0
X_1_4 = 0  at a cost of 0
X_1_5 = 0  at a cost of 0
X_1_6 = 0  at a cost of 0
X_1_7 = 0  at a cost of 0
X_2_0 = 1  at a cost of 13972000
X_2_1 = 1  at a cost of 3856000
X_2_2 = 0  at a cost of 0
X_2_3 = 1  at a cost of 17380000
X_2_4 = 0  at a cost of 0
X_2_5 = 0  at a cost of 0
X_2_6 = 0  at a cost of 0
X_2_7 = 1  at a cost of 4324000
X_3_0 = 0  at a cost of 0
X_3_1 = 0  at a cost of 0
X_3_2 = 0  at a cost of 0
X_3_3 = 0  at a cost of 0
X_3_4 = 0  at a cost of 0
X_3_5 = 0  at a cost of 0
X_3_6 = 0  at a cost of 0
X_3_7 = 0  at a cost of 0
X_4_0 = 0  at a cost of 0
X_4_1 = 0  at a cost of 0
X_4_2 = 1  at a cost of 3500000
X_4_3 = 0  at a cost of 0
X_4_4

In [52]:
import pandas as pd
import numpy as np



transport = pd.read_csv('/home/filipe/Documents/learning/supply_chain/SC2x_W2PP1_Estimating_costs_DATA_inbound.csv')
print(transport.head())
dc_costs = pd.read_csv('/home/filipe/Documents/learning/supply_chain/SC2x_W2PP1_Estimating_costs_DATA_dc_costs.csv')
print(dc_costs.head())


   Shipment ID    From  Number of widgets  Price
0            1  Site 3                  1     25
1            2  Site 1                  3     45
2            3  Site 3                  4    100
3            4  Site 3                 13    325
4            5  Site 3                  3     75
       Date  Number in stock  Daily Operating costs
0  8/1/2014              100                    236
1  8/2/2014               94                    237
2  8/3/2014               32                    123
3  8/4/2014               52                    136
4  8/5/2014               72                    171


In [63]:
for site in ['Site 1', 'Site 2', 'Site 3']:
    data = transport.loc[transport['From'].isin([site])]
    from sklearn import linear_model
    lr = linear_model.LinearRegression()
    y = data['Price']
    x = data['Number of widgets']
    lr.fit(X=x.values.reshape(-1, 1), y=y)
    print(site)
    print(np.sum(y)/np.sum(x))
    print('intercept', lr.intercept_)
    print('beta', lr.coef_)
    print('\n')

Site 1
10.506108202443281
intercept 17.354831729545367
beta [8.41625936]


Site 2
16.0
intercept -2.842170943040401e-14
beta [16.]


Site 3
25.0
intercept 8.526512829121202e-14
beta [25.]




In [64]:
lr = linear_model.LinearRegression()
y = dc_costs['Daily Operating costs']
x = dc_costs['Number in stock']
lr.fit(X=x.values.reshape(-1, 1), y=y)
print('intercept', lr.intercept_)
print('beta', lr.coef_)
print('\n')



intercept 42.28294760232606
beta [1.98437771]




In [ ]:
from ortools.constraint_solver import pywrapcp

solver = pywrapcp.Solver('solve')


cost = 1 # $ per mile per paper

rw_demands = [140,180,240,210,175,130,320,280,160]

distance_cw_to_rw =[[89,78,69,95,85,73,88,73,86],
                   [90,91,63,61,94,58,89,90,55],
                   [81,82,71,98,62,71,85,75,72],
                   [73,97,55,75,86,54,62,99,59],
                   [85,98,93,82,87,58,98,72,50]] #cw to rw

distance_plant_to_cw = [[77,48,41,57,94],
                        [54,33,21,94,44]]

n_plants = len(distance_plant_to_cw)
n_potential_cw = len(distance_plant_to_cw[0])
n_rw = len(distance_cw_to_rw[0])


cw_fixed_costs = [100000, 90000, 80000, 70000, 60000]
cw_variable_costs = [8, 10, 25, 30, 50]

p_capacities = [1500, 500]
p_variable_costs = [8, 13]


###########################
### FLOWS BETWEEN NODES ###
###########################

cw_boolean = [solver.IntVar(0,1,'cw{cw}'.format(cw=cw))\
              for cw in range(n_potential_cw)]


F_p_to_cw={}
for p in range(n_plants):
    for cw in range(n_potential_cw):
        F_p_to_cw[(p, cw)] = solver.IntVar(0, p_capacities[p], 'x_p{p}_cw{cw}'.format(p=p,cw=cw))\
        *cw_boolean[cw]
        

F_cw_to_rw={}
for cw in range(n_potential_cw):
    for rw in range(n_rw):
        F_cw_to_rw[(cw, rw)] = solver.IntVar(0, rw_demands[rw], 'x_cw{cw}_rw{rw}'.format(rw=rw,cw=cw))\
        *cw_boolean[cw]



####################
### TRAVEL COSTS ###
####################

p_to_cw_travel_costs = []
for p in range(n_plants):
    for cw in range(n_potential_cw):
        p_to_cw_travel_costs.append(F_p_to_cw[(p, cw)]*\
                                    distance_plant_to_cw[p][cw]*cost)

cw_to_rw_travel_costs = []
for cw in range(n_potential_cw):
    for rw in range(n_rw):
        cw_to_rw_travel_costs.append(F_cw_to_rw[(cw, rw)]*\
                                     distance_cw_to_rw[cw][rw]*cost)

########################
# plant variable costs #
########################
# flow out times variable costs
plant_variable_costs = [F_p_to_cw[(p, cw)]*p_variable_costs[p] \
                                   for p in range(n_plants) \
                                   for cw in range(n_potential_cw)]

#######################
## CW variable costs ##
#######################
cw_variable_costs = [F_p_to_cw[(p, cw)]*cw_variable_costs[cw] \
                                   for p in range(n_plants) \
                                   for cw in range(n_potential_cw)]
####################                             
## CW fixed costs ##
####################
cw_fixed_costs = [cw_boolean[cw]*cw_fixed_costs[cw] for cw in range(n_potential_cw)]


###################
### CONSTRAINTS ###
###################

## flow out of plants is no greater than their capacity
for p in range(n_plants):
        solver.Add(solver.Sum([F_p_to_cw[(p, cw)]\
                               for cw in range(n_potential_cw)]) <= p_capacities[p])

## flow into to rw is equal to their demand
for rw in range(n_rw):
    solver.Add(solver.Sum([F_cw_to_rw[(cw, rw)] \
                           for cw in range(n_potential_cw)]) == rw_demands[rw])
        

    
##### NOW need to add up all the total costs and 
TOTAL_costs = solver.Sum(plant_variable_costs + p_to_cw_travel_costs\
                         + cw_fixed_costs + cw_variable_costs + cw_to_rw_travel_costs)





### flatten the X
F_p_to_cw_flat = [F_p_to_cw[(p, cw)] \
                  for p in range(n_plants) \
                  for cw in range(n_potential_cw)]

F_cw_to_rw_flat = [F_cw_to_rw[(cw, rw)] \
                  for cw in range(n_potential_cw)\
                 for rw in range(n_rw)]


### Ths solution finder
objective = solver.Minimize(TOTAL_costs, 1)
decision_builder = solver.Phase(F_p_to_cw_flat + F_cw_to_rw_flat + cw_boolean,
                                solver.CHOOSE_FIRST_UNBOUND,
                                solver.ASSIGN_MIN_VALUE)
# Create a solution collector.
collector = solver.LastSolutionCollector()
# Add the decision variables.
collector.Add(F_p_to_cw_flat)
collector.Add(F_cw_to_rw_flat)

  
# Add the objecw_booleanctive.
collector.AddObjective(TOTAL_costs)
solver.Solve(decision_builder, [objective, collector])

if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    #for dc in range(n_dcs):
    #    for shelter in range(n_shelters):
    #        print('X_{dc}_{sh} ='.format(dc=dc, sh=shelter),
    #              collector.Value(best_solution, X[(dc,shelter)]),
    #             ' at a cost of', collector.Value(best_solution, travel_costs_dict[(dc,shelter)]))










In [23]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver('solve', 
                        pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


cost = 1 # $ per mile per paper

rw_demands = [140,180,240,210,175,130,320,280,160]

M = 10*sum(rw_demands)

distance_cw_to_rw =[[89,78,69,95,85,73,88,73,86],
                   [90,91,63,61,94,58,89,90,55],
                   [81,82,71,98,62,71,85,75,72],
                   [73,97,55,75,86,54,62,99,59],
                   [85,98,93,82,87,58,98,72,50]] #cw to rw

distance_plant_to_cw = [[77,48,41,57,94],
                        [54,33,21,94,44]]

n_plants = len(distance_plant_to_cw)
n_potential_cw = len(distance_plant_to_cw[0])
n_rw = len(distance_cw_to_rw[0])


cw_fixed_costs = [100000, 90000, 80000, 70000, 60000]
cw_variable_costs = [8, 10, 25, 30, 50]

p_capacities = [1500, 500]
p_variable_costs = [8, 13]

###########################
### FLOWS BETWEEN NODES ###
###########################

cw_boolean = [solver.IntVar(0,1,'cw{cw}'.format(cw=cw)) for cw in range(n_potential_cw)]


F_p_to_cw={}
for p in range(n_plants):
    for cw in range(n_potential_cw):
        F_p_to_cw[(p, cw)] = solver.IntVar(0, p_capacities[p], 'x_p{p}_cw{cw}'.format(p=p,cw=cw))#*cw_boolean[cw]
        

F_cw_to_rw={}
for cw in range(n_potential_cw):
    for rw in range(n_rw):
        F_cw_to_rw[(cw, rw)] = solver.IntVar(0, rw_demands[rw], 'x_cw{cw}_rw{rw}'.format(rw=rw,cw=cw))#*cw_boolean[cw]


####################
### TRAVEL COSTS ###
####################

p_to_cw_travel_costs = []
for p in range(n_plants):
    for cw in range(n_potential_cw):
        p_to_cw_travel_costs.append(F_p_to_cw[(p, cw)]*\
                                    distance_plant_to_cw[p][cw]*cost)

cw_to_rw_travel_costs = []
for cw in range(n_potential_cw):
    for rw in range(n_rw):
        cw_to_rw_travel_costs.append(F_cw_to_rw[(cw, rw)]*\
                                     distance_cw_to_rw[cw][rw]*cost)

########################
# plant variable costs #
########################
# flow out times variable costs
plant_variable_costs = [F_p_to_cw[(p, cw)]*p_variable_costs[p] \
                                   for p in range(n_plants) \
                                   for cw in range(n_potential_cw)]

#######################
## CW variable costs ##
#######################
cw_variable_costs = [F_p_to_cw[(p, cw)]*cw_variable_costs[cw] \
                                   for p in range(n_plants) \
                                   for cw in range(n_potential_cw)]
####################                             
## CW fixed costs ##
####################
cw_fixed_costs = [cw_boolean[cw]*cw_fixed_costs[cw] for cw in range(n_potential_cw)]


###################
### CONSTRAINTS ###
###################

## flow out of plants is no greater than their capacity
for p in range(n_plants):
        solver.Add(solver.Sum([F_p_to_cw[(p, cw)]\
                               for cw in range(n_potential_cw)]) <= p_capacities[p])


## flow into CW is equal to flow out of CW
for cw in range(n_potential_cw):
    f_in = solver.Sum([F_p_to_cw[(p, cw)] for p in range(n_plants)])
    f_out = solver.Sum([F_cw_to_rw[(cw, rw)] for rw in range(n_rw)])
    solver.Add(f_in==f_out)

        
        
## flow into to rw is equal to their demand
for rw in range(n_rw):
    solver.Add(solver.Sum([F_cw_to_rw[(cw, rw)] \
                           for cw in range(n_potential_cw)]) == rw_demands[rw])
        

for cw in range(n_potential_cw):
    solver.Add(solver.Sum([F_cw_to_rw[(cw, rw)] for rw in range(n_rw)]) - M*cw_boolean[cw] <= 0)
        

 

TOTAL_costs = solver.Sum(plant_variable_costs + p_to_cw_travel_costs\
                         + cw_fixed_costs + cw_variable_costs + cw_to_rw_travel_costs)



### flatten the X
F_p_to_cw_flat = [F_p_to_cw[(p, cw)] \
                  for p in range(n_plants) \
                  for cw in range(n_potential_cw)]

F_cw_to_rw_flat = [F_cw_to_rw[(cw, rw)] \
                  for cw in range(n_potential_cw)\
                 for rw in range(n_rw)]


### Ths solution finder
objective = solver.Minimize(TOTAL_costs)
solver.Solve()
print('done')



done


In [24]:
for p in range(n_plants):
    for cw in range(n_potential_cw):
        print(p, cw, F_p_to_cw[(p, cw)].SolutionValue())

0 0 0.0
0 1 1335.0
0 2 0.0
0 3 0.0
0 4 0.0
1 0 0.0
1 1 500.0
1 2 0.0
1 3 0.0
1 4 0.0


In [25]:
for cw in range(n_potential_cw):
    for rw in range(n_rw):
        print(cw, rw, F_cw_to_rw[(cw, rw)].SolutionValue())

0 0 0.0
0 1 0.0
0 2 0.0
0 3 0.0
0 4 0.0
0 5 0.0
0 6 0.0
0 7 0.0
0 8 0.0
1 0 140.0
1 1 180.0
1 2 240.0
1 3 210.0
1 4 175.0
1 5 130.0
1 6 320.0
1 7 280.0
1 8 160.0
2 0 0.0
2 1 0.0
2 2 0.0
2 3 0.0
2 4 0.0
2 5 0.0
2 6 0.0
2 7 0.0
2 8 0.0
3 0 0.0
3 1 0.0
3 2 0.0
3 3 0.0
3 4 0.0
3 5 0.0
3 6 0.0
3 7 0.0
3 8 0.0
4 0 0.0
4 1 0.0
4 2 0.0
4 3 0.0
4 4 0.0
4 5 0.0
4 6 0.0
4 7 0.0
4 8 0.0


In [ ]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver('solve', 
                        pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

#so_demand = [110, 80, 155, 175, 210, 130, 180, 160]
so_demand = [121, 96, 166, 175, 210, 130, 180, 160]

plant_capacity = [1250, 200]
p_unit_cost = [10, 50]
#(second plant is the contractrctor)

dc_fixed_costs_ = [100, 100, 80, 70, 60]
dc_unit_costs_ = [10, 10, 25, 30, 30]


distance_dc_to_so = [[62,55,114,106,51,76,122,80],
                     [63,92,90,50,88,47,136,47],
                     [76,85,56,65,129,40,89,100],
                     [42,128,74,70,45,56,81,131],
                     [77,135,118,59,124,93,41,94]]

distance_p_to_dc = [[77,48,41,57,94],
                    [54,33,21,94,44]]
#(second plant is the contractor)

transport_costs = 1


n_plants = len(plant_variable_costs)
n_dcs = len(dc_fixed_costs)
n_sos = len(so_demand)


plant_boolean = [solver.IntVar(0,1, 'p{p}'.format(p=p)) for p in range(n_plants)]
dc_boolean = [solver.IntVar(0,1, 'dc{dc}'.format(dc=dc)) for dc in range(n_dcs)]

F_p_to_dc = {}
for p in range(n_plants):
    for dc in range(n_dcs):
        F_p_to_dc[(p, dc)] = solver.IntVar(0, plant_capacity[p],
                                           'f_p{p}_dc{dc}'.format(p=p, dc=dc))

F_dc_to_so={}
for cw in range(n_dcs):
    for so in range(n_sos):
        F_dc_to_so[(dc, sp)] = solver.IntVar(0, so_demand[so],
                                             'x_dc{dc}_so{so}'.format(dc=dc, so=so))#*cw_boolean[cw]


####################
### TRAVEL COSTS ###
####################

p_to_dc_travel_costs = []
for p in range(n_plants):
    for dc in range(n_dcs):
        p_to_dc_travel_costs.append(F_p_to_dc[(p, dc)]*\
                                    distance_p_to_dc[p][dc]*transport_costs)

dc_to_so_travel_costs = []
for dc in range(n_dcs):
    for so in range(n_sos):
        dc_to_so_travel_costs.append(F_dc_to_so[(dc, so)]*\
                                     distance_dc_to_so[dc][so]*transport_costs)

########################
# plant  costs #
########################
# flow out times variable costs
plant_variable_costs = [F_p_to_cw[(p, cw)]*p_variable_costs[p] \
                                   for p in range(n_plants) \
                                   for dc in range(n_dcs)]



########################
# dc variable costs #
########################
# flow out times variable costs
dc_variable_costs = [F_dc_to_so[(dc, so)]*dc_unit_costs_[dc] \
                                   for dc in range(n_dcs) \
                                   for so in range(n_sos)]

dc_fixed_costs = [dc_boolean[p]*dc_fixed_costs_ for dc in range(n_dcs)]




